<a href="https://colab.research.google.com/github/deeptiishwar/CIP/blob/main/cnn_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [6]:
INIT_LR = 1e-4  # Initial Learning rate
EPOCHS = 20     # Number of epochs to train
BS = 32         # Batch size

# Data paths
train_dir = '/content/drive/MyDrive/data/train'
val_dir = '/content/drive/MyDrive/data/test'


In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")

val_datagen = ImageDataGenerator(rescale=1./255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=BS,
    class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=BS,
    class_mode='binary')

# Load MobileNetV2
baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

Found 2637 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


9406464/9406464 [==============================] - 0s 0us/step


In [8]:
# Construct head of the model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)

# Place the head FC model on top of the base model
model = Model(inputs=baseModel.input, outputs=headModel)

# Freeze the base model layers
for layer in baseModel.layers:
    layer.trainable = False

# Compile the model
# Compile the model with Adam optimizer
opt = Adam(learning_rate=INIT_LR)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=EPOCHS)

# Unfreeze some layers of the base model for fine-tuning
for layer in baseModel.layers[100:]:
    layer.trainable = True



Epoch 1/20
83/83 [==============================] - 540s 6s/step - loss: 0.6192 - accuracy: 0.6553 - val_loss: 0.4754 - val_accuracy: 0.7788
Epoch 2/20
83/83 [==============================] - 207s 2s/step - loss: 0.4994 - accuracy: 0.7691 - val_loss: 0.4599 - val_accuracy: 0.7758
Epoch 3/20
83/83 [==============================] - 220s 3s/step - loss: 0.4567 - accuracy: 0.7865 - val_loss: 0.4053 - val_accuracy: 0.8258
Epoch 4/20
83/83 [==============================] - 213s 3s/step - loss: 0.4334 - accuracy: 0.8077 - val_loss: 0.3890 - val_accuracy: 0.8258
Epoch 5/20
83/83 [==============================] - 217s 3s/step - loss: 0.4234 - accuracy: 0.8085 - val_loss: 0.3798 - val_accuracy: 0.8455
Epoch 6/20
83/83 [==============================] - 229s 3s/step - loss: 0.4115 - accuracy: 0.8149 - val_loss: 0.3708 - val_accuracy: 0.8500
Epoch 7/20
83/83 [==============================] - 215s 3s/step - loss: 0.4053 - accuracy: 0.8111 - val_loss: 0.3642 - val_accuracy: 0.8485
Epoch 8/20
83

In [10]:
from tensorflow.keras.optimizers.legacy import Adam as AdamLegacy
# Re-compile the model (necessary after modifications to the trainability)

model.compile(loss="binary_crossentropy", optimizer=AdamLegacy(learning_rate=INIT_LR, decay=INIT_LR / (EPOCHS * 10)), metrics=["accuracy"])
# Continue training (fine-tuning)
fine_tune_epochs = 10
total_epochs = EPOCHS + fine_tune_epochs
history_fine = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    epochs=total_epochs,
    initial_epoch=history.epoch[-1])

Epoch 20/30
83/83 [==============================] - 296s 3s/step - loss: 0.4417 - accuracy: 0.7933 - val_loss: 2.1745 - val_accuracy: 0.5621
Epoch 21/30
83/83 [==============================] - 288s 3s/step - loss: 0.3388 - accuracy: 0.8506 - val_loss: 1.8412 - val_accuracy: 0.5682
Epoch 22/30
83/83 [==============================] - 278s 3s/step - loss: 0.2985 - accuracy: 0.8692 - val_loss: 1.9150 - val_accuracy: 0.5879
Epoch 23/30
83/83 [==============================] - 274s 3s/step - loss: 0.2555 - accuracy: 0.8889 - val_loss: 2.2548 - val_accuracy: 0.5848
Epoch 24/30
83/83 [==============================] - 276s 3s/step - loss: 0.2431 - accuracy: 0.8938 - val_loss: 1.3918 - val_accuracy: 0.6727
Epoch 25/30
83/83 [==============================] - 275s 3s/step - loss: 0.2064 - accuracy: 0.9162 - val_loss: 1.4891 - val_accuracy: 0.6667
Epoch 26/30
83/83 [==============================] - 275s 3s/step - loss: 0.2163 - accuracy: 0.9063 - val_loss: 1.7655 - val_accuracy: 0.6318
Epoch 

In [11]:
import tensorflow as tf


# Save the fine-tuned model in SavedModel format
fine_tuned_model_dir = '/content/drive/MyDrive/data/hi'
print(type(model))
model.save(fine_tuned_model_dir)

# Convert the fine-tuned SavedModel to TFLite model
converter = tf.lite.TFLiteConverter.from_saved_model(fine_tuned_model_dir)
tflite_model = converter.convert()

# Save the TFLite model to a file
tflite_model_file = 'fine_tuned_model_malignant_benign_classification.tflite'
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)

print("Fine-tuned TFLite model saved to:", tflite_model_file)



<class 'keras.src.engine.functional.Functional'>
Fine-tuned TFLite model saved to: fine_tuned_model_malignant_benign_classification.tflite
